In [1]:
import numpy as np
import tensorflow as tf
import argparse
import os
from six.moves import cPickle
import nltk
import time
import queue as Q
from operator import itemgetter
from model_back import Model as Model_back
from model_forw import Model as Model_forw

import itertools
from numpy.random import choice
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
import string
import re
import collections
import sys
import pickle
import getopt
import random
from collections import defaultdict, Counter
import itertools
counts = defaultdict(Counter)
from Traversal import *
from functions import *
from tensorflow.python.framework import ops

c:\users\hp\anaconda2\envs\translator\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
c:\users\hp\anaconda2\envs\translator\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#LOAD DIRECTORY OF MODELS
text_list = [("data\\all_combined\\input.txt","all_combined_forward"),("data\all_combined\input.txt","all_combined_back")]
#np.random.shuffle(text_list)
t = text_list[0][0] #THIS TEXT IS THE VOCAB!
save_dir = text_list[0][1] #THIS IS THE MODEL DIRECTORY
t_back=text_list[1][0]
save_dir_back=text_list[1][1]

width = 20
wordPools = [set([]) for n in range(4)]

In [3]:
# load glove to a gensim model
glove_model = KeyedVectors.load_word2vec_format('./storyline_for_reference/glove.6B.300d.word2vec.txt',binary=False)

# system arguments
topics = [sys.argv[1]]
try:
    seed = int(sys.argv[2])
except:
    seed = 1
    
text = open(t)
text = text.read()

np.random.seed(seed) # seed for reproducibility
words = [simple_clean(word) for word in text.split()]
uniques = set()
for word in words:
    uniques.add(word)
corpus = list(uniques)
dictWordTransitions = {}
glove_words = glove_model.vocab.keys() #SHOULDN'T THIS HAVE BEEN MOST COMMON WORDS IN THE CORPUS?
# CORPUS LIMITED TO WORDS IN GLOVE VOCAB
word_counts = collections.Counter([x for x in words])
corpi = [x[0] for x in word_counts.most_common() if x[0] in glove_words]
dictPartSpeechTags = createPartSpeechTags(corpi)
dictPossiblePartsSpeech = possiblePartsSpeechPaths()
vowels = ["AA", "AE", "AH", "AO", "AW", "AY", "EH", "ER", "EY", "IH", "IY", "OW", "OY", "UH", "UW"]
width = 20
wordPools = [set([]) for n in range(4)]


In [4]:
def genPoem_forward(save_dir,width, wordPools, word_last, TemplatePOS):
    start = time.time()
    def sampleLine(lst, cut):
        ''' samples from top "cut" lines, the distribution being the softmax of true sentence probabilities'''
        probs = list()
        for i in range(cut):
            probs.append(np.exp(lst[i][0][0][0]))
        probs = np.exp(probs) / sum(np.exp(probs))
        index = np.random.choice(cut,1,p=probs)[0]
        return lst[index][1][1]
    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_forw(saved_args, True)
    print (len(vocab))
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            state = sess.run(model.initial_state)
            init_score = np.array([[0]])
            lst = search_forward(model, vocab, init_score,[word_last],state, sess, 1,\
                              dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, TemplatePOS)
            lst.sort(key=itemgetter(0), reverse = True)
            # line diagnostics
            #for i in range(10)
            #    print(lst[i][0][0][0], lst[i][1][1])
            #choice = sampleLine(lst, min(10,len(lst)))
            #print (choice)
            #poem.append(choice)
            #line_num+=1
            #if(line_num>3):
            #    line_num = 0
            #    wordPool_ind+=1
        #poem = postProcess(poem,sess,vocab,model)
    print("Generation took {:.3f} seconds".format(time.time() - start))
    return lst





In [5]:
def genPoem_backward(save_dir,width, wordPools, word_last, TemplatePOS):
    start = time.time()
    def sampleLine(lst, cut):
        ''' samples from top "cut" lines, the distribution being the softmax of true sentence probabilities'''
        probs = list()
        for i in range(cut):
            probs.append(np.exp(lst[i][0][0][0]))
        probs = np.exp(probs) / sum(np.exp(probs))
        index = np.random.choice(cut,1,p=probs)[0]
        return lst[index][1][1]
    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    print(len(vocab))
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            state = sess.run(model.initial_state)
            init_score = np.array([[0]])
            lst = search_back(model, vocab, init_score,[word_last],state, sess, 1,\
                              dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, TemplatePOS)
            lst.sort(key=itemgetter(0), reverse = True)
            # line diagnostics
            #for i in range(10)
            #    print(lst[i][0][0][0], lst[i][1][1])
            #choice = sampleLine(lst, min(10,len(lst)))
            #print (choice)
            #poem.append(choice)
            #line_num+=1
            #if(line_num>3):
            #    line_num = 0
            #    wordPool_ind+=1
        #poem = postProcess(poem,sess,vocab,model)
    print("Generation took {:.3f} seconds".format(time.time() - start))
    return lst

In [6]:
def force_middle(save_dir, width, lst, next_word):
    def decayRepeat(word,sequence, scale):
        safe_repeat_words = []
        #safe_repeat_words = set(["with,the,of,in,i"])
        score_adjust = 0
        decr = -scale
        for w in range(len(sequence)):
            if(word==sequence[w] and word not in safe_repeat_words):
                score_adjust += decr
            decr += scale/10 #decreases penalty as the words keep getting further from the new word
        return score_adjust
    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_forw(saved_args, True)
    print (len(vocab))
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            ret=[]
            for tup in lst:
                state_init = tup[1][0][1]
                prob_seq =  tup[1][0][0]
                sequence=tup[1][1]
                scale = .02 #scale is the significant magnitude required to affect the score of bad/good things
                dist, state = model.compute_fx(sess, vocab, prob_seq, sequence, state_init, 1)
                #PREVENTS REPEAT ADJACENT WORDS OR PROBLEM-TAGGED WORDS
                if(next_word == sequence[-1]):
                    continue
                #if(partsOfSpeechFilter(sequence[-1],next_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
                #    continue
                #FACTORS IN SCORE ADJUSTMENTS
                score_adjust = decayRepeat(next_word, sequence, 100*scale) #repeats
                score_adjust += scale*len(next_word)/50 #length word
                #if(next_word in wordPool):
                #    score_adjust += scale
                #CALCULATES ACTUAL SCORE
                key = np.array([[vocab[next_word]]])
                new_prob = dist[key]
                score_tuple = (new_prob, state)
                score_tup = (score_tuple[0]+score_adjust,score_tuple[1]) #NOTE SHOULD SCORE_ADJUST BE ADDED HERE OR JUST IN THE ITEM LINE?
                item = (score_tup[0],(score_tup, sequence+[next_word]))
                if(item[0]==[[-float("inf")]]):
                    continue
                ret+=[item]
            ret.sort(key=itemgetter(0), reverse = True)
            return ret[:150]

In [7]:
#LOAD DICTIONARIES CREATED IN EXTRAXTING_POSTAG.IPYNB
with open('postag_dict_all.p','rb') as f:
    postag_dict=pickle.load(f)
PartOfSpeachSet=postag_dict[1]
TemplatePOS=['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

In [8]:
postag_dict[0].keys()

dict_keys(['NN-NN', 'JJ-RB', 'RB-RB', 'JJ-VB', 'JJ-JJ', 'NN-NNS', 'NN-VB', 'NNS-VB', 'NN-RB', 'NNS-RB', 'RB-VB', 'JJ-NN', 'JJ-NNS', 'VB-VB', 'NNS-NNS'])

In [9]:
#function to get a template based on two words
def pos_synset(words, pos_dict):
    postag_nn=[]
    print(words)
    for word in words:
        pos_word=nltk.pos_tag([word])[0][1]#nltk.pos_tag(word)[0]#postag_dict[2][word][0]
        postag_nn.append(pos_word)
    print (postag_nn)
    pos=str(postag_nn[0])+'-'+str(postag_nn[1])
    if pos in set(list(pos_dict.keys())):
        possible_templates=pos_dict[pos]
        possible=[k_ for k_ in possible_templates if len(k_[0])<8]
    else:
        pos=str(postag_nn[1])+'-'+str(postag_nn[0])
        if pos in set(list(pos_dict.keys())):
            possible_templates=pos_dict[pos]
            possible=[k_ for k_ in possible_templates if len(k_[0])<8]
            #postag_nn=[postag_nn[1], postag_nn[0]]
            words=[words[1], words[0]]
        else:
            return None
    template_list=list(random.choice(possible))
    if postag_nn[0]==postag_nn[1]:
        template_list.append(words)
        return template_list
    else:
        ww=[words[postag_nn.index(postag_nn[0])], words[postag_nn.index(postag_nn[1])]]
        template_list.append(ww)
        return template_list

## combine

In [10]:
def generate_line(word1, word2, postag_=None):
    nouns=[word1, word2]

    postag_nn=[]
    #SELECT A POS SAMPLE
    """for noun in nouns:
        postag_nn.append(nltk.pos_tag([noun])[0][1])
    if postag_nn.count('NN')>1:
        postag=list(random.choice(postag_dict[0]['NN-NN']))
        nouns=nouns
        postag.append(nouns)
    elif postag_nn.count('NN')==1 and postag_nn.count('NNS')==1:
        postag=list(random.choice(postag_dict[0]['NN-NNS']))
        nouns=[nouns[postag_nn.index('NN')], nouns[postag_nn.index('NNS')]]
        postag.append(nouns)
    elif postag_nn.count('NNS')>1:
        postag=list(random.choice(postag_dict[0]['NNS-NNS']))
        nouns=nouns
        postag.append(nouns)
    else:
        print ('WORDS ARE NOT NOUNS')
        #return None
        """
    if postag_==None:
        postag=pos_synset(nouns, postag_dict[0])
    else:
        postag=postag_
    if postag==None:
        return None
    print (postag)
    #postag=[['CC', 'PRP', 'RB', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN'], [3,6], ['night', 'moon']]
    print (postag[0])
    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])

    
    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]
    
    first_pos=postag[1][first_position]
    last_pos=postag[1][last_position]
    template=postag[0]
    print ('First noun: '+ str(first_noun))
    print ('Last noun: '+ str(last_noun))
    print (template[first_pos:last_pos])

    print ('#################################\n')

    list_1= genPoem_forward(save_dir,width,wordPools,first_noun, template[first_pos:last_pos])
    #print ('show' in [tup[1][1][1] for tup in list_1])
    print (list_1[0][1][1])
    #print (list_1[0][1][0][1])
    list_2=force_middle(save_dir, 100, list_1, last_noun)
    print (list_1[0][1][1])

    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_forw(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_3=[]
            for element in list_2:
                seq=element[1][1]
                state = element[1][0][1]
                init_score = element[1][0][0]
                seq=element[1][1]
                lst = search_forward(model, vocab, init_score,seq,state, sess, 1,\
                                  dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, template[first_pos:])
                tt_3+=lst
            tt_3.sort(key=itemgetter(0), reverse = True)
           
    tf.reset_default_graph()
    with open(os.path.join(save_dir_back, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir_back, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir_back)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_4=[]
            for tup in tt_3[:10]:
                seq=tup[1][1]
                init_score, state=model.score_a_list(sess, vocab, seq)
                lst = search_back(model, vocab, init_score,seq,state, sess, 1,\
                              dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, template)
                
                tt_4+=lst
    tt_4.sort(key=itemgetter(0), reverse = True)
    
                
    return template, tt_3, tt_4

In [214]:
seq=["the","dark", "secret"]
template=["JJ", "NN","VBZ","DT","JJ","NN"]
tf.reset_default_graph()
with open(os.path.join(save_dir_back, 'config.pkl'), 'rb') as f:
    saved_args = cPickle.load(f)
with open(os.path.join(save_dir_back, 'words_vocab.pkl'), 'rb') as f:
    word_keys, vocab = cPickle.load(f)
model = Model_back(saved_args, True)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(save_dir_back)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        init_score, state=model.score_a_list(sess, vocab, seq)
        lst = search_back(model, vocab, init_score,seq,state, sess, 0.2,\
                              dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, template)

INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  """Returns log p(y+x), and state after prediction,"""


In [215]:
lst[0][1][1]

['great', 'death', 'is', 'the', 'dark', 'secret']

In [100]:
poem=genPoem_forward(save_dir, 20, wordPools, 'feelings', ['NNS','VB','NNS'])

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 15.616 seconds


In [106]:
print ('*************\n')
#print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[5][1][1]))

*************


feelings make men


In [30]:
t=generate_line('eyes', 'feelings', postag_=(['NNS', 'VB','NNS'],[0,2],['eyes','feelings']))

print ('*************\n')
#print ('POS: ', TemplatePOS)

print('\n'+' '.join(t[1][0][1][1]))

(['NNS', 'VB', 'NNS'], [0, 2], ['eyes', 'feelings'])
['NNS', 'VB', 'NNS']
First noun: eyes
Last noun: feelings
['NNS', 'VB']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 6.481 seconds
['eyes', 'look']
25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
['eyes', 'look']
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000
*************


eyes look feelings


In [157]:
t=generate_line('stars', 'sky', postag_=(["DT",'NNS', 'VB',"IN","DT", 'NN'],[1,5],["stars","sky"]))

print ('*************\n')
#print ('POS: ', TemplatePOS)

print('\n'+' '.join(t[2][0][1][1]))

(['DT', 'NNS', 'VB', 'IN', 'DT', 'NN'], [1, 5], ['stars', 'sky'])
['DT', 'NNS', 'VB', 'IN', 'DT', 'NN']
First noun: stars
Last noun: sky
['NNS', 'VB', 'IN', 'DT']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 6.932 seconds
['stars', 'sleep', 'in', 'the']
25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
['stars', 'sleep', 'in', 'the']
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000
*************


the stars sleep in the sky


## poem in meeting

In [238]:
#to travel the whole world in journeys

t=generate_line('recalling', 'memories', postag_=(['DT',
   'NN',
   'VBZ',
   'RB',
   'DT',
   'NN',
   ',',
   'PRP',
   'VBZ',
   'DT',
   'NN',
   'IN',
   'DT',
   'NN',
   ','],[1,10],["compassion","sympathy"]))

print ('*************\n')
#print ('POS: ', TemplatePOS)

print('\n'+' '.join(t[2][0][1][1]))

(['DT', 'NN', 'VBZ', 'RB', 'DT', 'NN', ',', 'PRP', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN', ','], [1, 10], ['compassion', 'sympathy'])
['DT', 'NN', 'VBZ', 'RB', 'DT', 'NN', ',', 'PRP', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN', ',']
First noun: compassion
Last noun: sympathy
['NN', 'VBZ', 'RB', 'DT', 'NN', ',', 'PRP', 'VBZ', 'DT']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 15.878 seconds
['compassion', 'does', 'not', 'any', 'doubt', ',', 'it', 'is', 'the']
25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
['compassion', 'does', 'not', 'any', 'doubt', ',', 'it', 'is', 'the']
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000
*************


a compassion does not any doubt , it is the sympathy of a man ,


In [242]:
print('\n'+' '.join(t[2][10][1][1]))


that compassion does not any doubt , it is the sympathy of a man ,


In [244]:
#and trip before adventorous journeys

t=generate_line('trip', 'journeys', postag_=(['PRP',
   'VBP',
   'VBN',
   'WP',
   'DT',
   'NNS',
   'VBD',
   'VBG',
   ',',
   'DT',
   'NN',
   'IN',
   'DT',
   'NN'],[10,13],["ornament","colour"]))

print ('*************\n')
#print ('POS: ', TemplatePOS)

print('\n'+' '.join(t[2][0][1][1]))

(['PRP', 'VBP', 'VBN', 'WP', 'DT', 'NNS', 'VBD', 'VBG', ',', 'DT', 'NN', 'IN', 'DT', 'NN'], [10, 13], ['ornament', 'colour'])
['PRP', 'VBP', 'VBN', 'WP', 'DT', 'NNS', 'VBD', 'VBG', ',', 'DT', 'NN', 'IN', 'DT', 'NN']
First noun: ornament
Last noun: colour
['NN', 'IN', 'DT']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 11.298 seconds
['ornament', 'while', 'the']
25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
['ornament', 'while', 'the']
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000
*************


i have heard what the birds were sinking , the ornament as a colour


In [245]:

t=generate_line('journey', 'shame', postag_=(["DT", 'JJ',"NN","IN", "PRP$","NN"],[2,5],["colour","sorrow"]))

print ('*************\n')
#print ('POS: ', TemplatePOS)

print('\n'+' '.join(t[2][0][1][1]))

(['DT', 'JJ', 'NN', 'IN', 'PRP$', 'NN'], [2, 5], ['colour', 'sorrow'])
['DT', 'JJ', 'NN', 'IN', 'PRP$', 'NN']
First noun: colour
Last noun: sorrow
['NN', 'IN', 'PRP$']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 8.708 seconds
['colour', 'onto', 'his']
25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
['colour', 'onto', 'his']
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000
*************


the red colour below his sorrow


# Results

# one word

In [44]:
TemplatePOS=['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

## input first word

In [27]:
poem=genPoem_forward(save_dir, 20, wordPools, 'he', TemplatePOS)

print ('*************\n')
print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[0][1][1]))

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 26.623 seconds
*************

POS:  ['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

he is a man going to the ground


## input last word

In [42]:
poem=genPoem_backward(save_dir_back, 20, wordPools, 'darkness', TemplatePOS)

print ('*************\n')
print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[0][1][1]))

25676
INFO:tensorflow:Restoring parameters from try_load\model.ckpt-120000
Generation took 15.084 seconds
*************

POS:  ['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

it is a look coming to another darkness


In [41]:
poem=genPoem_backward(save_dir_back, 20, wordPools, 'human', TemplatePOS)

print ('*************\n')
print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[0][1][1]))

25676
INFO:tensorflow:Restoring parameters from try_load\model.ckpt-120000
Generation took 22.928 seconds
*************

POS:  ['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

it is a look coming to each human


In [105]:
TemplatePOS=['PRP', 'VBP', 'DT', 'NN', 'CC', 'VBD', 'PRP', 'RP', 'JJ', 'NNS']
poem=genPoem_backward(save_dir_back, 20, wordPools, 'people', TemplatePOS)

print ('*************\n')
print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[1][1][1]))

13969
INFO:tensorflow:Restoring parameters from whitman_model\model.ckpt-433199
Generation took 6.369 seconds
*************

POS:  ['PRP', 'VBP', 'DT', 'NN', 'CC', 'VBD', 'PRP', 'RP', 'JJ', 'NNS']

i take this day and gave him down new people


### Poem in meeting

# Evaluate different templates

### memories and old

In [199]:
l_vocab=len(vocab)
import pandas as pd
temp_score=[]
for x in range(6):
    try:
        template, _,k=generate_line('story','old')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        n=len(k[1][1])
        t=k[0]
        p=np.log(1/(l_vocab**n))
        r=t-p
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),t/n, template) 
        temp_score.append(row)
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

['story', 'old']
['NN', 'JJ']
[['NN', 'VBZ', 'JJ'], [0, 2], ['story', 'old']]
['NN', 'VBZ', 'JJ']
First noun: story
Last noun: old
['NN', 'VBZ']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 6.289 seconds
['story', 'is']
25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
['story', 'is']
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000
['story', 'old']
['NN', 'JJ']
[['NN', 'VBZ', 'DT', 'JJ', 'NN', 'RB'], [0, 3], ['story', 'old']]
['NN', 'VBZ', 'DT', 'JJ', 'NN', 'RB']
First noun: story
Last noun: old
['NN', 'VBZ', 'DT']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 6.750 seconds
['story', 'is', 'a']
25676
INFO:tensorflow:Restoring parameters from all_combined_forw

,line,score,POS
0,story is old,-4.91561,"[NN, VBZ, JJ]"
1,story is the old man there,-7.30015,"[NN, VBZ, DT, JJ, NN, RB]"
2,story is old,-4.91561,"[NN, VBZ, JJ]"
3,the story old fellow,[[-4.940456]],"[DT, NN, VBZ, JJ]"
4,story is old man,-5.44634,"[NN, VBZ, JJ, NN]"
5,with the old story of men,[[-4.1209292]],"[IN, DT, JJ, NN, IN, NNS]"


In [ ]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

In [198]:
postag_dict[2]["old"]

['JJ']

In [200]:
sentence=[["IN", "DT", "JJ", "NN", "IN", "NNS"],
          ["DT","NN","VBZ"],
          ["DT","NNS","VBP"],
          ["NN","VBZ"],
          ["NNS","VBP"],
          ["DT","NN","VBZ","NN"],
          ["DT","NN","VBZ","THE","NN"],
          ["NNS","VB","PRP"],
          ["NNP","VBZ","JJ"],
          ["PRP","VBZ","JJ"],
          ["PRP","VBP","JJ"],
          ["NN","VBZ","RB"],
          ["NNS","VB","RB"],
          ["PRP","VB","DT","NNS"],
          ["PRP","RB","VB","NNS","DT","NN"],
          ["NN","VBZ","DT","JJ","NN","RB"],
          ["NNS","VB","RB"],
          ["JJ",",","JJ","NNS","VB","RB"],
          ["NN","VBZ","JJ","NN"]]

DEF={"NN-NNS":[(["IN", "DT", "JJ", "NN", "IN", "NNS"],[3,5]), 
              (["PRP","RB","VB","NNS","DT","NN"],[5,3])],
    "NN-NN":[(["DT","NN","VBZ","NN"],[1,3]),
            (["DT","NN","VBZ","THE","NN"],[1,4]),
            (["NN","VBZ","DT","JJ","NN","RB"],[0,4]),
            (["NN","VBZ","JJ","NN"],[0,3])],
    "NN-JJ":[(["IN", "DT", "JJ", "NN", "IN", "NNS"],[3,2]),
             (["NN","VBZ","JJ"],[0,2]),
             (["DT","NN","VBZ","JJ"],[1,3]),
             (["NN","VBZ","DT","JJ","NN","RB"],[0,3]),
             (["NN","VBZ","JJ","NN"],[0,2]),
             (["NN","VBZ","JJ","DT","NN"],[0,2]),
             (["NN","VBZ","JJ","IN","DT","NN"],[0,2])],
    "NNS-VB":[(["DT","NNS","VB"],[1,2]),( ["NNS","VB","PRP"],[0,1]),( ["NNS","VB","RB"],[0,1]),(["PRP","VB","DT","NNS"],[3,1]),
              ( ["PRP","RB","VB","NNS","DT","NN"],[3,2]),(["NNS","VB","RB"],[0,1]),(["JJ",",","JJ","NNS","VB","RB"],[3,4])
        
    ]}
postag_dict[0]=DEF

### bring and light

In [202]:
import pandas as pd
temp_score=[]
for x in range(8):
    try:
        template,_, k=generate_line('bring','hopes')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0]/len(k[1][1]), template) 
        temp_score.append(row)
print (' ...')

['bring', 'hopes']
['NN', 'NNS']
[['PRP', 'RB', 'VB', 'NNS', 'DT', 'NN'], [5, 3], ['bring', 'hopes']]
['PRP', 'RB', 'VB', 'NNS', 'DT', 'NN']
First noun: hopes
Last noun: bring
['NNS', 'DT']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 6.021 seconds
['hopes', 'the']
25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
['hopes', 'the']
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000
['bring', 'hopes']
['NN', 'NNS']
[['IN', 'DT', 'JJ', 'NN', 'IN', 'NNS'], [3, 5], ['bring', 'hopes']]
['IN', 'DT', 'JJ', 'NN', 'IN', 'NNS']
First noun: bring
Last noun: hopes
['NN', 'IN']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 6.267 seconds
['bring', 'like']
25676
INFO:tensorflow

In [203]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,i still have hopes that bring,[[-4.772673]],"[PRP, RB, VB, NNS, DT, NN]"
1,of the modern bring like hopes,[[-4.478604]],"[IN, DT, JJ, NN, IN, NNS]"
2,i still have hopes that bring,[[-4.772673]],"[PRP, RB, VB, NNS, DT, NN]"
3,i still have hopes that bring,[[-4.772673]],"[PRP, RB, VB, NNS, DT, NN]"
4,of the modern bring like hopes,[[-4.478604]],"[IN, DT, JJ, NN, IN, NNS]"
5,of the modern bring like hopes,[[-4.478604]],"[IN, DT, JJ, NN, IN, NNS]"
6,of the modern bring like hopes,[[-4.478604]],"[IN, DT, JJ, NN, IN, NNS]"
7,i still have hopes that bring,[[-4.772673]],"[PRP, RB, VB, NNS, DT, NN]"


### climb stairs

In [207]:
import pandas as pd
temp_score=[]
for x in range(8):
    try:
        template,_, k=generate_line('climb','stairs')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0]/len(k[1][1]), template) 
        temp_score.append(row)
print (' ...')

['climb', 'stairs']
['NN', 'NNS']
[['IN', 'DT', 'JJ', 'NN', 'IN', 'NNS'], [3, 5], ['climb', 'stairs']]
['IN', 'DT', 'JJ', 'NN', 'IN', 'NNS']
First noun: climb
Last noun: stairs
['NN', 'IN']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 6.132 seconds
['climb', 'in']
25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
['climb', 'in']
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
INFO:tensorflow:Restoring parameters from all_combined_back\model.ckpt-120000
['climb', 'stairs']
['NN', 'NNS']
[['IN', 'DT', 'JJ', 'NN', 'IN', 'NNS'], [3, 5], ['climb', 'stairs']]
['IN', 'DT', 'JJ', 'NN', 'IN', 'NNS']
First noun: climb
Last noun: stairs
['NN', 'IN']
#################################

25676
INFO:tensorflow:Restoring parameters from all_combined_forward\model.ckpt-114000
Generation took 5.940 seconds
['climb', 'in']
25676
INFO:tensorflow:

In [208]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,with a little climb out stairs,[[-4.981931]],"[IN, DT, JJ, NN, IN, NNS]"
1,with a little climb out stairs,[[-4.981931]],"[IN, DT, JJ, NN, IN, NNS]"
2,with a little climb out stairs,[[-4.981931]],"[IN, DT, JJ, NN, IN, NNS]"
3,with a little climb out stairs,[[-4.981931]],"[IN, DT, JJ, NN, IN, NNS]"
4,i just go stairs the climb,[[-4.612494]],"[PRP, RB, VB, NNS, DT, NN]"
5,i just go stairs the climb,[[-4.612494]],"[PRP, RB, VB, NNS, DT, NN]"
6,with a little climb out stairs,[[-4.981931]],"[IN, DT, JJ, NN, IN, NNS]"
7,i just go stairs the climb,[[-4.612494]],"[PRP, RB, VB, NNS, DT, NN]"


##  eyes emotion

In [206]:
import pandas as pd
temp_score=[]
for x in range(8):
    try:
        template,_, k=generate_line('emotions','eyes')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0]/len(k[1][1]), template) 
        temp_score.append(row)
print (' ...')

['emotions', 'eyes']
['NNS', 'NNS']


TypeError: 'NoneType' object is not iterable

In [92]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS


In [242]:
import pandas as pd
temp_score=[]
for x in range(8):
    try:
        template, k=generate_line('time', 'sound')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0]/len(k[1][1]), template) 
        temp_score.append(row)
print (' ...')

['time', 'sound']
['NN', 'NN']
['IN', 'IN', 'NN', 'CC', 'NN']
First noun: time
Last noun: sound
['NN', 'CC']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 10.165 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['time', 'sound']
['NN', 'NN']
['DT', 'NN', 'NN']
First noun: time
Last noun: sound
['NN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 5.927 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['time', 'sound']
['NN', 'NN']
['PRP$', 'NN', 'VBZ', 'NN']
First noun: sound
Last noun: time
['NN', 'VBZ']
##

C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


['time', 'sound']
['NN', 'NN']
['CC', 'IN', 'NN', 'VBG', 'PDT', 'DT', 'NN']
First noun: sound
Last noun: time
['NN', 'VBG', 'PDT', 'DT']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 5.888 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['time', 'sound']
['NN', 'NN']
['NN', 'RB', ',', 'CC', 'IN', 'IN', 'NN']
First noun: sound
Last noun: time
['NN', 'RB', ',', 'CC', 'IN', 'IN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
 ...


In [243]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,beside that time and sound,[[-4.153561]],"[IN, IN, NN, CC, NN]"
1,the time sound,[[-4.6519017]],"[DT, NN, NN]"
2,her sound is time,[[-4.3387413]],"[PRP$, NN, VBZ, NN]"
3,things to the time sound,[[-4.418531]],"[NNS, TO, DT, NN, NN]"
4,or time in sound taking new hampshire,[[-7.205396]],"[CC, NN, IN, NN, VBG, NNP, NNP]"
5,the long time then made that sound,[[-4.7765946]],"[DT, JJ, NN, RB, VBN, IN, NN]"
6,and whats sound standing half the time,[[-5.2065086]],"[CC, IN, NN, VBG, PDT, DT, NN]"


In [263]:
'express' in PartOfSpeachSet['VB']

True